#### Import Julia packages

In [ ]:
# using Pkg
# Pkg.add("PyPlot")


    Updating registry at `C:\Users\Bruker\.julia\registries\General.toml`
   Resolving package versions...
   Installed PyPlot ─ v2.11.6
   Installed PyCall ─ v1.96.4
    Updating `C:\Users\Bruker\.julia\environments\v1.11\Project.toml`
  [d330b81b] + PyPlot v2.11.6
    Updating `C:\Users\Bruker\.julia\environments\v1.11\Manifest.toml`
  [438e738f] + PyCall v1.96.4
  [d330b81b] + PyPlot v2.11.6
    Building PyCall → `C:\Users\Bruker\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`
Precompiling project...
  14092.8 ms  ✓ PyCall
   4273.6 ms  ✓ PyPlot
  2 dependencies successfully precompiled in 36 seconds. 385 already precompiled.


In [24]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot

┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall C:\Users\Bruker\.julia\packages\PyCall\1gn3u\src\PyCall.jl:719
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda C:\Users\Bruker\.julia\packages\Conda\zReqD\src\Conda.jl:181


Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: C:\Users\Bruker\.julia\conda\3\x86_64

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            2_gnu          48 KB  conda-forge
    contourpy-1.3.1            |  py312h214f63a_0         233 KB
    cycler-0.11.0              |     pyhd3eb1b0_0          12 KB
    fonttools-4.55.3           |  py312h827c3e9_0         2.6 MB
    freetype-2.13.3            |       h0b5ce68_0         499 KB  conda-forge
    glib-2.82.2                |       h3d4babf_1         560 KB  conda-forge
    glib-tools-2.82.2          |       h4394cf3_1          95 KB  conda-forge
    gst-plugins-base-1.24.7    |       hb0a98b8_0         2.0 MB  conda-forge
    gstreamer-1.24.7           |       h5006eae_0         1.9 MB  conda-forge
    icu-75.1                   |       he0c23c2_0        13.9 MB 

#### Import Custom functions

In [29]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")

result_plot (generic function with 1 method)

#### Domain & Discretization

In [ ]:
domain = (-1,+1,-1,+1)
partition = (20,20)
model = CartesianDiscreteModel(domain,partition) |> simplexify

order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                          ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions

Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                             # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)                                                       # make the measure dΩ
Γ = BoundaryTriangulation(model)                                             # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)                                                       # measure on Γ

#TODO: Legg opp til ulik rand for vinduer, dører og vegger

GenericMeasure()

#### Time parameters

In [4]:
t0 = 0.0    # Start time
tF = 10.0   # End time
Δt = 0.05   # Timestep

0.05

#### Room parameters

In [ ]:
ρ(x)=1.225
c(x)=1020.0
k(x)=15.0
h(x)=0.7

Toutdoor(x,t)=5.0
Tout(t)=x->Toutdoor(x,t)

price(t)=1.0

Tini(x)=20.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 800
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 14656515611062853333

#### Controll parameter

In [6]:
q_pos(x) = χ(x[1], -0.9, -0.7) * χ(x[2], 0, 0.20) + χ(x[1], 0.2, 0.4) * χ(x[2], -0.5, 0)
power = 10.0
Q(x,t)=χ(t,0.0,10.0)*power*q_pos(x)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [9]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)
ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)
γ = 1

Proj(a,b,z) = min(max(a,z),b)
# Proj(z) = map(x->Proj(a,b,x),z)
a=0.0
b=1000.0
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]
# [(t,interpolate_everywhere(map(x->P(a,b,x),get_free_dof_values(zz)),Uspace(t))) for (t,zz) in z]


Proj (generic function with 2 methods)

#### Optimize

In [ ]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")

(ys,qs,ps,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=dΓ, 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=true, 
tol=1e-5, 
iter_max=2, 
Δt=Δt, 
t0=t0, 
tF=tF)

q computed
y computed
p computed
entered for loop, E=33.24222006350655
s_min = -0.4999961178237603
entered for loop, E=3.1123427598850617e9
s_min = -0.4999995873335625


(Vector{Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}}[[(0.0, SingleFieldFEFunction()), (0.05, SingleFieldFEFunction()), (0.1, SingleFieldFEFunction()), (0.15000000000000002, SingleFieldFEFunction()), (0.2, SingleFieldFEFunction()), (0.25, SingleFieldFEFunction()), (0.3, SingleFieldFEFunction()), (0.35, SingleFieldFEFunction()), (0.39999999999999997, SingleFieldFEFunction()), (0.44999999999999996, SingleFieldFEFunction())  …  (9.55, SingleFieldFEFunction()), (9.600000000000001, SingleFieldFEFunction()), (9.650000000000002, SingleFieldFEFunction()), (9.700000000000003, SingleFieldFEFunction()), (9.750000000000004, SingleFieldFEFunction()), (9.800000000000004, SingleFieldFEFunction()), (9.850000000000005, SingleFieldFEFunction()), (9.900000000000006, SingleFieldFEFunction()), (9.950000000000006, SingleFieldFEFunction()), (10.000000000000007, SingleFieldFEFunction())], [(0.0, SingleFieldFEFunction()), (0.05, SingleFieldFEFunction()), (0.1, SingleFieldFEFunction(

#### Results

In [19]:
#TODO: lag plot; pris, pådrag, temperatur 

In [31]:
result_plot()

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\Bruker\.julia\packages\PyCall\1gn3u\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("type object 'FigureManagerBase' has no attribute '_owns_mainloop'")
  File "C:\Users\Bruker\.julia\conda\3\x86_64\Lib\site-packages\matplotlib\pyplot.py", line 614, in show
    return _get_backend_mod().show(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bruker\.julia\conda\3\x86_64\Lib\site-packages\matplotlib\backend_bases.py", line 3547, in show
    cls.mainloop()
  File "C:\Users\Bruker\.julia\conda\3\x86_64\Lib\site-packages\matplotlib\backends\_backend_tk.py", line 540, in start_main_loop
    if manager_class._owns_mainloop:
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
